## Assignment 1
# Group 13
Mathieu Mailhot - Isabel Lougheed - Frank-Lucas Pantazis

Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os


In [ ]:
"""
- CKD: 28 numerical features, 1 target binary classification variable ("Normal" / "battery")
- Battery: 32 real-valued features, 2 classes ("Normal" / "Defective")
"""

# load data sets

# Calculate cross entropy or/ Information Gain for all the data without the threshold

# statistical analysis on the datasets

# - normalize

# models: all features, selective features based on statistical analysis (dropping features)


'\n- CKD: 28 numerical features, 1 target binary classification variable ("Normal" / "CKD")\n- Battery: 32 real-valued features, 2 classes ("Normal" / "Defective")\n'

Statistics

In [5]:
# load data sets
df_CKD = pd.read_csv("CKD.csv")
df_battery = pd.read_csv("Battery_Dataset.csv")

# Convert "CKD" to 1 and "Normal" to 0
df_CKD["label"] = df_CKD["label"].replace({"CKD": 1, "Normal": 0})
# Convert "Defective" to 1 and "Normal" to 0
df_battery["label"] = df_battery["label"].replace({"Defective": 1, "Normal": 0})

# Convert to a numpy array
CKD_data = df_CKD.to_numpy()
battery_data = df_battery.to_numpy()

In [ ]:
# Statistical Analysis Block

# Class for the analysis
class Stat_analysis:
    def __init__(self, data, name, save_folder):
        self.data = data
        self.name = name
        self.save_folder = save_folder
        self.feature_distribution()
        self.class_distrubution()

    # Function to create a distribution for each feature
    def feature_distribution(self):
        for i in range(self.data.shape[1] - 2): # remove 1 and last column as we do not need them for the distribution of the features
            feature_num = i + 1

            plt.hist(self.data[:,feature_num], bins=20, edgecolor="black")
            plt.xlabel("Value")
            plt.ylabel("Frequency")
            plt.title(f"{self.name} Distribution of Feature {feature_num}")

            filename = os.path.join(self.save_folder, f"{self.name}_feature{feature_num}_distribution.png")
            plt.savefig(filename, dpi=300, bbox_inches="tight")
            plt.close()
    
    # Function to create a distribution for the class
    def class_distrubution(self):
        my_bins = [-0.5, 0.5, 1.5]
        class_0 = "Normal"
        if self.name == "CKD":
            class_1 = "CKD"
        else:
            class_1 = "Defective"

        plt.hist(self.data[:,self.data.shape[1] - 1], bins=my_bins, edgecolor="black", align="mid", rwidth=0.6)
        plt.xticks([0, 1], [class_0, class_1])
        plt.xlabel("Category")
        plt.ylabel("Frequency")
        plt.title(f"{self.name} Distribution of Class")

        filename = os.path.join(self.save_folder, f"{self.name}_class_distribution.png")
        plt.savefig(filename, dpi=300, bbox_inches="tight")
        plt.close()

# Perform the stastical analysis
CKD_stat = Stat_analysis(CKD_data, "CKD", "CKD_distribution")
battery_stat = Stat_analysis(battery_data, "Battery", "Battery_distribution")




In [ ]:
class Model:
    def __init__(self,  dataframe, data_array, description:str = None):
        
        # Hyperparameter variables for Adam Gradient Descent Algorithm
        self.max_iteration = 10000
        self.tolerance = 10**-3
        self.step_size = 0.01
        self.b_1 = 0.93
        
        # Data variables
        self.dataframe = dataframe
        self.data_array = data_array
        self.description = description
        
        # Training and Validation variables
        self.accuracy_arr = []
        self.avg_accuracy = 0
        
        self.weigth_arr = []
        self.avg_weigth = 0
    
    def normalize(self, normalize_by_max:bool, standardize:bool): 
        # Normalize the dataset
        # I think you should only normalize by max OR standardize, and I think standardizing would produce better results

        # Separate features and target
        df = self.dataframe # make sure data is a data frame
        features = df.drop(['ID', 'label'], axis=1)
        target = df['label']

        df_norm = df # if normalize_by_max = false and standardize = false, will return original df

        if normalize_by_max:
            # normalizing by extremas, scales to [0,1]
            # ensures data is well-conditioned
            features_normalized = (features - features.min())/(features.max() - features.min())
            df_norm = pd.concat([df[['ID']], features_normalized, df[['label']]], axis=1)

        if standardize:
            # z score normalization, good for gaussian distributions
            # forces std 1 and mean 0
            features_standardized = (features - features.mean())/features.std()
            df_norm = pd.concat([df[['ID']], features_standardized, df[['label']]], axis=1)

        # returns a pandas dataframe
        return df_norm
    
    def crossValidation(self, folds:int): 
        # Split dataset into folds
        # I think that self.data should only include non test data
        data = self.data_array[:,1:] # removing first column (ID)
        fold_size = len(data) // folds
        validation_experiments = []
        train_experiments = []

        for i in range(folds):
            if i==(folds-1):
                # how should i deal with uneven split ??? is it okay for the last fold to be smaller?
                validation_fold = data[(i*fold_size):]  #df.iloc[(i*fold_size):(len(df))]
                train_fold = data[:(i*fold_size)] #pd.concat([df.iloc[:(i*fold_size)], df.iloc[(len(df)):]])

            else:
                validation_fold = data[(i*fold_size):(i*fold_size + fold_size)] #df.iloc[(i*fold_size):(i*fold_size + fold_size)]
                train_fold = np.vstack([data[:(i*fold_size)], data[(i*fold_size + fold_size):]]) #pd.concat([df.iloc[:(i*fold_size)], df.iloc[(i*fold_size + fold_size):]])
            validation_experiments.append(validation_fold)
            train_experiments.append(train_fold)

        # Train
        avg_error1 = 0
        avg_error2 = 0
        np.array
        for i in range(folds):
            # train each training set with fit() to get weights
            train_experiment = train_experiments[i]
            w = self.fit(train_experiment)
            self.weigth_arr.append(w.tolist())
            # get errors 
            validation_experiment = validation_experiments[i]
            error1,error2 = self.Accu_eval(w, validation_experiment) 
            
            avg_error1 += error1
            avg_error2 += error2
        
        self.avg_weigth = np.mean(np.array(self.weigth_arr),axis = 0)
        
  
        avg_error1 = avg_error1/folds
        avg_error2 = avg_error2/folds
        return (avg_error1,avg_error2)

    def fit(self,train_data):
        
        # Trains using gradient descent: Lecture 5 slide 55-58

        w_prev = np.ones(train_data.shape[1])*0.1 # (number of columns of train_data - 1 to remove label, + 1 to account for bias term)
        m = 0
        for epoch in range(self.max_iteration):
            delta = np.zeros(len(w_prev))

            for row_i in train_data:
                y_i = row_i[-1] # extracting value
                x_i = row_i.copy() # extracting features
                x_i[-1] = 1 # because of bias term
                delta += x_i*(y_i-self.predict(w_prev,x_i))

            m = self.b_1 * m + (1-self.b_1) * delta
            w_new = w_prev + self.step_size * m # to be able to reach better precision need to devide step size by num iteration
       
            w_diff = np.linalg.norm(w_new - w_prev)
            
            w_prev = w_new
            
            if (w_diff**2<self.tolerance):
                return w_new
           
            
        # Means there was a problem
        print("Not converged", w_new , w_diff)
        return w_new

    
    def predict(self,w,x):
        # Predicts output: function at bottom Lecture 5 slide 44
        a = w.T @ x
        return 1/(1+np.exp(-np.clip(a, -500, 500)))

    def Accu_eval(self,w,validate_data): # Used MSE
        # Validation
        error = 0
        correct_prediction = 0
        for row_i in validate_data:
            y_i = row_i[-1] # extracting value
            x_i = row_i.copy() # extracting features
            x_i[-1] = 1
            
            error += (y_i-self.predict(w,x_i))**2 
            if ((y_i-self.predict(w,x_i))<=0.5):
                correct_prediction+=1
            
        return (error/len(validate_data), 1-correct_prediction/len(validate_data))



In [ ]:
# testing whole model for CKD data with cross validation


ckd_model = Model(df_CKD, CKD_data, "This is the model for the CKD dataset")
features_ckd = df_CKD.drop(['ID', 'label'], axis=1)

def condition_number(features):
    A = features.to_numpy()
    U, S, V = np.linalg.svd(A, full_matrices=False)
    condition_num = np.max(S) / np.min(S[np.nonzero(S)])
    return condition_num

# normalize data
df_norm = ckd_model.normalize(True, False)
features_ckd_norm = df_norm.drop(['ID', 'label'], axis=1)
ckd_model_norm = Model(df_norm, df_norm.to_numpy())

# standardize data
df_stand = ckd_model.normalize(False, True)
features_ckd_stand = df_stand.drop(['ID', 'label'], axis=1)
ckd_model_stand = Model(df_stand, df_stand.to_numpy())

#print("Condition number of original dataset : ", condition_number(features_ckd))
print("Condition number of normalized dataset : ", condition_number(features_ckd_norm))
print("Condition number of standardized dataset : ", condition_number(features_ckd_stand))
print("---------------------------------------------------------")
#print("Error after training original dataset : ", ckd_model.crossValidation(10))
print("Error after training normalized dataset : ", ckd_model_norm.crossValidation(10), ckd_model_norm.weigth_arr)

print("Error after training standardized dataset : ", ckd_model_stand.crossValidation(10),ckd_model_stand.avg_weigth)


Condition number of normalized dataset :  16.272627194785652
Condition number of standardized dataset :  1.6816260377340286
---------------------------------------------------------
Error after training normalized dataset :  (0.210624438089218, 0.16363636363636364) [[-1.388483900520639, 5.378751515011773, -0.5122352533582972, -0.2583136879542226, -0.18983052878476656, -0.02974513986772246, -0.282649543144278, -1.1644466814414245, -0.9067668160423416, -0.40623361015646436, 0.23650155960519312, 0.7524265471395645, -0.13614778002974506, -0.28506762754678294, -0.7245981182278461, -0.16316134613449917, 0.36180120997761034, 0.24124775206975163, 0.4072821098162617, 0.35412890574298805, 0.3163026357272578, -1.0432217522485654, -0.5509806290536292, 0.40672245481357666, -0.28893174224284074, 0.16940716747135887, 0.34518117192083625, 0.2527877726780731, -0.2857297162604279], [-1.6195990571363437, 4.959709199176352, -0.08214205281584519, -0.026097158674979307, -0.11932896032843217, 0.0357572095026

In [431]:
battery_model = Model(df_battery, battery_data, "This is the model for the battery dataset")
features_battery = df_battery.drop(['ID', 'label'], axis=1)

def condition_number(features):
    A = features.to_numpy()
    U, S, V = np.linalg.svd(A, full_matrices=False)
    condition_num = np.max(S) / np.min(S[np.nonzero(S)])
    return condition_num

# normalize data
df_norm = battery_model.normalize(True, False)
features_battery_norm = df_norm.drop(['ID', 'label'], axis=1)
battery_model_norm = Model(df_norm, df_norm.to_numpy())

# standardize data
df_stand = battery_model.normalize(False, True)
features_battery_stand = df_stand.drop(['ID', 'label'], axis=1)
battery_model_stand = Model(df_stand, df_stand.to_numpy())

#print("Condition number of original dataset : ", condition_number(features_battery))
print("Condition number of normalized dataset : ", condition_number(features_battery_norm))
print("Condition number of standardized dataset : ", condition_number(features_battery_stand))
print("---------------------------------------------------------")
#print("Error after training original dataset : ", battery_model.crossValidation(10))
print("Error after training normalized dataset : ", battery_model_norm.crossValidation(10))
print("Error after training standardized dataset : ", battery_model_stand.crossValidation(10))

Condition number of normalized dataset :  17.632795820381226
Condition number of standardized dataset :  1.6891992219130847
---------------------------------------------------------
Error after training normalized dataset :  (0.0860907227531073, 0.060091324200913274)
Error after training standardized dataset :  (0.08748855983724914, 0.06009132420091327)
